In [1]:
import json
import os
import torch
import numpy as np
import transformers as tfs
import pandas as pd
from tqdm import tqdm
tqdm.pandas(desc='pandas bar')
from torch import nn
import joblib


C:\Users\aron\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
softmax = nn.Softmax(dim=1)
model_path = 'model'
dataset_path = 'dataset'
SUMMARY_OUTPUT_PATH = 'summary'
PRETRAINED_BERT_ENCODER_PATH = 'pretrain'
FINETUNED_BERT_ENCODER_PATH = os.path.join(model_path, "finetuned_bert.bin")
BERT_MODEL_SAVE_PATH = model_path
PU_MODEL_SAVE_PATH = os.path.join(model_path, "pu_model.bin")
TEST_FILE_PATH = os.path.join(dataset_path, "preprocessed_test.json")
SUMMARY_OUTPUT_PATH = os.path.join(SUMMARY_OUTPUT_PATH, "submission.csv")


In [ ]:
INDEX = INDEX = ['人物专栏', '作品分析', '情感解读', '推荐文', '攻略文', '治愈系文章', '深度事件', '物品评测', '科普知识文', '行业解读']
MODEL_EPOCH=5


class MyBertEncoder(nn.Module):
    """自定义的Bert编码器"""
    def __init__(self, tokenizer_path, finetuned_bert_path):
        super(MyBertEncoder, self).__init__()
        model_class, tokenizer_class = tfs.BertModel, tfs.BertTokenizer
        self.tokenizer = tokenizer_class.from_pretrained(tokenizer_path)
        self.bert = torch.load(finetuned_bert_path)

    def forward(self, batch_sentences):
        batch_tokenized = self.tokenizer.batch_encode_plus(batch_sentences, add_special_tokens=True,
                                                           max_length=512, pad_to_max_length=True)

        input_ids = torch.tensor(batch_tokenized['input_ids']).cuda()
        token_type_ids = torch.tensor(batch_tokenized['token_type_ids']).cuda()
        attention_mask = torch.tensor(batch_tokenized['attention_mask']).cuda()

        bert_output = self.bert(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        bert_cls_hidden_state = bert_output[0][:, 0, :]
        return bert_cls_hidden_state
def prepare_sequence(title: str, body: str):
    return (title, body[:256] + "|" + body[-256:])
def read_test_file(input_path:str):
    test_df = pd.read_json(input_path,orient="records",lines=True)
    
    return test_df
def predict_with_pu(x,idnex,pu_classifier,bert_encoder,bert_classifier_model):
    text = prepare_sequence(x["title"],x["body"])
    encoded_pos = np.array(bert_encoder([text]).tolist())
    pu_result = pu_classifier(encoded_pos)
    if pu_result[0] < 0:
        predicted_label = "其他"
        proba = 0.5
    else:
        output = bert_classifier_model([text])
        predicted_proba = softmax(output).tolist()[0]
        predicted_index = np.argmax(predicted_proba)
        predicted_label = index[predicted_index]
        
        proba = predicted_proba[predicted_index]
    return [predicted_label,round(prob,2)]
def summary(test_df,output_path,pu_classifier,bert_encoder,bert_classifier_model):
    test_df[["preidcted_label","proba"]] = test_df.progress_apply(
     lambda x:pd.Series(predict_with_pu(x, INDEX, pu_classifier, bert_encoder, bert_classifier_model)), axis=1)
    csv_data = test_df.loc[:,["id,predicted_label"]]
    csv_data.columns = ["id","predict_doctype"]
    csv_data.to_csv(output_path,index=0,line_terminator="\r\r\n")
def joint_predictor():
    torch.cuda.set_device(0)
    fs = os.listdir(BERT_MODEL_SAVE_PATH)
    gs = list()
    for f in fs:
        if 'model_epoch' in f:
            gf.append(f)
    MODEL_EPOCH = max([int(x.split('.')[0].split('model_epoch')[-1])] for x in gs)
    model_save_path = os.path.join(BERT_MODEL_SAVE_PATH,'model_epoch{}.pkl'.format(MODEL_EPOCH))
    test_df = read_test_file(TEST_FILE_PATH)
    bert_classifier_model = torch.load(model_save_path)
    bert_classifier_model = bert_classifier_model.cuda()
    bert_classifier_model.evall()
    
    with torch.no_grad():
        pu_classifier = joblib.load(PU_MODEL_SAVE_PATH)
        bert_encpder = MyBertEncoder(PRETRAINED_BERT_ENCODER_PATH,FINETUNED_BERT_ENCODER_PATH)
        bert_encpder.eval()
        summary(test_df,SUMMARY_OUTPUT_PATH,pu_classifier,bert_encpder,bert_classifier_model)